In [100]:
import numpy as np
from mp_api.client import MPRester
from pymatgen.core.operations import SymmOp
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.electronic_structure import bandstructure
from pymatgen.electronic_structure.plotter import BSPlotter
from lmapr1492 import plot_brillouin_zone, get_plot_bs, get_plot_dos, get_plot_bs_and_dos, get_branch_wavevectors, latex_fix
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from matplotlib import pyplot as plt 

In [101]:
mp_key = "kPLAfUWZpfzd6GRd9caSEcAaWKf3DxYL"
mp_id = "mp-569677"

In [102]:
with MPRester(mp_key) as m:
    prim_struc = m.get_structure_by_material_id(mp_id)
    el_bs = m.get_bandstructure_by_material_id(mp_id)
    el_dos = m.get_dos_by_material_id(mp_id)
conv_struc = SpacegroupAnalyzer(prim_struc).get_conventional_standard_structure()
symmops = SpacegroupAnalyzer(conv_struc).get_space_group_operations()

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving ElectronicStructureDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

# Première zone de Brillouin

In [98]:
import itertools
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Plotting of the Brillouin zone
def go_points(points, size=4, color="black", labels=None):
    mode = "markers" if labels is None else "markers+text"

    if labels is not None:
        for il in range(len(labels)):
            labels[il] = latex_fix(labels[il])

    import plotly.graph_objects as go
    return go.Scatter3d(
        x=[v[0] for v in points],
        y=[v[1] for v in points],
        z=[v[2] for v in points],
        marker=dict(size=size, color=color),
        mode=mode,
        text=labels,
        textfont_color=color,
        showlegend=False
    )

def go_line(v1, v2, color="black", width=2, mode="lines", text=""):
    import plotly.graph_objects as go
    return go.Scatter3d(
        mode=mode,
        x=[v1[0], v2[0]],
        y=[v1[1], v2[1]],
        z=[v1[2], v2[2]],
        line=dict(color=color),
        text=text,
        showlegend=False
    )

In [99]:
struc = el_bs.structure
bz_lattice=struc.lattice.reciprocal_lattice
bz = bz_lattice.get_wigner_seitz_cell()
fig = go.Figure()
for iface in range(len(bz)):  # pylint: disable=C0200
        for line in itertools.combinations(bz[iface], 2):
            for jface in range(len(bz)):
                if (iface < jface
                    and any(np.all(line[0] == x) for x in bz[jface])
                    and any(np.all(line[1] == x) for x in bz[jface])):
                    fig.add_trace(go_line(line[0], line[1]))
fig.update_layout(
    scene = dict(
        xaxis = dict(visible=False, range=[-1.15,1.15],),
        yaxis = dict(visible=False, range=[-1.15,1.15],),
        zaxis = dict(visible=False, range=[-1.15,1.15],),
    )
)
fig.show()

In [93]:
plot_brillouin_zone(el_bs.structure)

# Bande électronique

In [104]:
fig_el_bs = get_plot_bs(el_bs, plot_range=[-2,4])
fig_el_bs.show()

**1) Analyse de la bande interdite**

In [105]:
# Calcul de la bande interdite
band_gap = el_bs.get_band_gap()
print(f"Bande interdite : {band_gap['energy']} eV")
print(f"Gap direct : {band_gap['direct']}")
print(f"Transition : {band_gap['transition']}")

Bande interdite : 1.7923 eV
Gap direct : False
Transition : F-\Gamma


**NB** : Dans ce cas-ci, Gamma correspond à $\hat{I}^{n}$

Comme la bande d énergie interdite vaut 1.7923, mon matériau est un semi-isolant. 

**2) Dispersions maximale et minimale sur la dernière bande de valence et la première bande de conduction**

In [106]:
# Récupération des valeurs des bandes
xvals = fig_el_bs.to_dict()["data"][0]["x"]
yvals_vbm = fig_el_bs.to_dict()["data"][4]["y"]  # Bande 16
yvals_cbm = fig_el_bs.to_dict()["data"][5]["y"]  # Bande 17

In [107]:
# Identification des indices des bandes de valence et conduction
vbm_indices = el_bs.get_vbm()["kpoint_index"]
cbm_indices = el_bs.get_cbm()["kpoint_index"]

In [108]:
# Calcul de la dispersion pour la bande de valence et de conduction
branch_indices = [(b["start_index"], b["end_index"]) for b in el_bs.branches]

vbm_dispersion = []
cbm_dispersion = []

for start, end in branch_indices:
    vbm_dispersion.append(max([yvals_vbm[i] for i in range(start, end+1)]) - min([yvals_vbm[i] for i in range(start, end+1)]))
    cbm_dispersion.append(max([yvals_cbm[i] for i in range(start, end+1)]) - min([yvals_cbm[i] for i in range(start, end+1)]))

In [109]:
# Identifier les indices des dispersions maximales et minimales
max_vbm_idx = branch_indices[np.argmax(vbm_dispersion)]
min_vbm_idx = branch_indices[np.argmin(vbm_dispersion)]
max_cbm_idx = branch_indices[np.argmax(cbm_dispersion)]
min_cbm_idx = branch_indices[np.argmin(cbm_dispersion)]

In [110]:
# Ajout des flèches indiquant la dispersion maximale et minimale
fig_el_bs.add_trace(go.Scatter(
    x=[xvals[max_vbm_idx[0]], xvals[max_vbm_idx[1]]],
    y=[yvals_vbm[max_vbm_idx[0]], yvals_vbm[max_vbm_idx[1]]],
    mode="lines+markers", line=dict(color="red", width=3),
    name="Max Dispersion VBM (Bande 16)"))

fig_el_bs.add_trace(go.Scatter(
    x=[xvals[min_vbm_idx[0]], xvals[min_vbm_idx[1]]],
    y=[yvals_vbm[min_vbm_idx[0]], yvals_vbm[min_vbm_idx[1]]],
    mode="lines+markers", line=dict(color="green", width=3),
    name="Min Dispersion VBM (Bande 16)"))

fig_el_bs.add_trace(go.Scatter(
    x=[xvals[max_cbm_idx[0]], xvals[max_cbm_idx[1]]],
    y=[yvals_cbm[max_cbm_idx[0]], yvals_cbm[max_cbm_idx[1]]],
    mode="lines+markers", line=dict(color="blue", width=3),
    name="Max Dispersion CBM (Bande 17)"))

fig_el_bs.add_trace(go.Scatter(
    x=[xvals[min_cbm_idx[0]], xvals[min_cbm_idx[1]]],
    y=[yvals_cbm[min_cbm_idx[0]], yvals_cbm[min_cbm_idx[1]]],
    mode="lines+markers", line=dict(color="purple", width=3),
    name="Min Dispersion CBM (Bande 17)"))

fig_el_bs.update_layout(xaxis_range=[xvals[0], xvals[-1]])
fig_el_bs.show()


**3) Calcul des masses effectives**

In [117]:
# Calcul de la masse effective en supposant une dispersion parabolique
hbar2 = ((6.62607004*10**(-34))/(2*np.pi))**2
electronvolt = 1.602176634*10**(-19)
angstrom = 10**(-10) 

In [118]:
ks_vbm = [xvals[i] for i in vbm_indices[-2:]] + [xvals[vbm_indices[0] - 1]]
es_vbm = [yvals_vbm[i] for i in vbm_indices[-2:]] + [yvals_vbm[vbm_indices[0] - 1]]

ks_cbm = [xvals[i] for i in cbm_indices[:2]] + [xvals[cbm_indices[-1] + 1]]
es_cbm = [yvals_cbm[i] for i in cbm_indices[:2]] + [yvals_cbm[cbm_indices[-1] + 1]]

a_vbm, _, _ = np.polyfit(np.array(ks_vbm), es_vbm, 2)
a_cbm, _, _ = np.polyfit(np.array(ks_cbm), es_cbm, 2)

/tmp/ipykernel_2009443/4228186019.py:7: RankWarning:

Polyfit may be poorly conditioned



In [119]:
m_eff_vbm = hbar**2 / (2 * a_vbm * electronvolt*(angstrom)**2)
m_eff_cbm = hbar**2 / (2 * a_cbm * electronvolt*(angstrom)**2)

print("Masse effective au sommet de la bande de valence:", m_eff_vbm)
print("Masse effective à la base de la bande de conduction:", m_eff_cbm)

Masse effective au sommet de la bande de valence: 3.793114404297888e-27
Masse effective à la base de la bande de conduction: 1.5378310015041937e-29
